In [1]:
# --- Celda 1: Configuración ---
import sagemaker
from sagemaker.pytorch import PyTorchModel
import boto3

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()

print("Configuración lista.")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Configuración lista.


In [2]:
# --- Celda 2: Recuperar la Ruta del Mejor Modelo ---
%store -r model_artifacts_s3_path

if 'model_artifacts_s3_path' not in locals():
    raise NameError("No se encontró 'model_artifacts_s3_path'. "
                    "Por favor, ejecuta la Celda 5 del notebook 'train_classifier.ipynb' para guardarla.")

print(f"Artefactos del modelo a desplegar: {model_artifacts_s3_path}")

Artefactos del modelo a desplegar: s3://sagemaker-us-east-1-318399884747/credit-risk-hpo-v7-cpu-final-002-048de319/output/model.tar.gz


In [3]:
# --- Celda 3: Crear el Objeto del Modelo de SageMaker ---
pytorch_model = PyTorchModel(
    model_data=model_artifacts_s3_path,
    role=role,
    entry_point='inference.py', 
    source_dir='./',           
    framework_version='2.0.0',
    py_version='py310'
)

print("Objeto PyTorchModel creado.")

Objeto PyTorchModel creado.


In [4]:
# --- Celda 4: Desplegar el Modelo a un Endpoint ---
instance_type = 'ml.m5.large' 
endpoint_name = 'credit-risk-classifier-endpoint-final' 

print(f"Iniciando despliegue en un endpoint de tipo '{instance_type}'. Esto puede tardar entre 5 y 10 minutos...")

predictor = pytorch_model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    endpoint_name=endpoint_name
)

print(f"\n¡Éxito! Endpoint '{endpoint_name}' desplegado.")

Iniciando despliegue en un endpoint de tipo 'ml.m5.large'. Esto puede tardar entre 5 y 10 minutos...
-------!
¡Éxito! Endpoint 'credit-risk-classifier-endpoint-final' desplegado.


In [6]:
# --- Celda 5: Probar el Endpoint con un Ejemplo Nuevo 
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer


predictor.serializer = JSONSerializer()
predictor.deserializer = JSONDeserializer()

# EJEMPLO 1: Un perfil que suena a 'mal riesgo'
test_description_bad = "The 29-year-old female customer, currently unemployed and living with relatives, reflects a high credit risk profile. Her lack of steady income, minimal savings, frequent overdrafts in her checking account, and recent application for multiple short-term loans suggest significant financial vulnerability."
print(f"\nProbando con perfil de 'mal riesgo': \"{test_description_bad}\"")
response_bad = predictor.predict(data={'description': test_description_bad})
print("Respuesta del endpoint:", response_bad)


Probando con perfil de 'mal riesgo': "The 29-year-old female customer, currently unemployed and living with relatives, reflects a high credit risk profile. Her lack of steady income, minimal savings, frequent overdrafts in her checking account, and recent application for multiple short-term loans suggest significant financial vulnerability."
Respuesta del endpoint: {'prediction': 'bad risk', 'confidence': 0.11438444256782532}


In [7]:
# EJEMPLO 2: Un perfil que suena a 'buen riesgo'
test_description_good = "The 38-year-old male customer, a full-time government employee and homeowner, demonstrates a strong credit risk profile. His consistent salary, substantial savings, long-standing checking account, and history of timely repayments on previous loans all point toward a reliable financial standing, making him a low-risk borrower."
print(f"\nProbando con perfil de 'buen riesgo': \"{test_description_good}\"")
response_good = predictor.predict(data={'description': test_description_good})
print("Respuesta del endpoint:", response_good)


Probando con perfil de 'buen riesgo': "The 38-year-old male customer, a full-time government employee and homeowner, demonstrates a strong credit risk profile. His consistent salary, substantial savings, long-standing checking account, and history of timely repayments on previous loans all point toward a reliable financial standing, making him a low-risk borrower."
Respuesta del endpoint: {'prediction': 'good risk', 'confidence': 0.8997997641563416}


In [ ]:
# --- Celda 6: Limpieza ---

#predictor.delete_endpoint()
#print(f"Endpoint '{endpoint_name}' eliminado.")

In [11]:
#test
print('''python predict.py "Example." --endpoint-name "credit-risk-classifier-endpoint-final"''')

python predict.py "Example." --endpoint-name "credit-risk-classifier-endpoint-final"
